In [1]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('Churn').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/27 19:41:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
df = spark.read.csv("/Users/eslinkiran/Apps/Pyspark-Tutorial/Logestic/customer_churn.csv", header=True, inferSchema= True)

In [6]:
df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [7]:
df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [8]:
from pyspark.ml.feature import VectorAssembler

In [10]:
assembler = VectorAssembler(inputCols=['Age','Total_Purchase','Account_Manager','Years','Num_Sites'], outputCol='features')

In [11]:
output = assembler.transform(df)

In [13]:
final_data = output.select('features','Churn')

In [14]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [16]:
from pyspark.ml.classification import LogisticRegression

In [17]:
model = LogisticRegression(labelCol='Churn')

In [18]:
fit_model = model.fit(train_data)

23/12/27 19:52:39 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/12/27 19:52:39 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


In [20]:
training_sum = fit_model.summary

In [22]:
training_sum.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              Churn|         prediction|
+-------+-------------------+-------------------+
|  count|                650|                650|
|   mean|0.17692307692307693|0.12615384615384614|
| stddev| 0.3818974728169655| 0.3322783669166222|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



In [23]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [24]:
pred_and_lable = fit_model.evaluate(test_data)

In [25]:
pred_and_lable.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|Churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[22.0,11254.38,1....|    0|[4.26555710727226...|[0.98615046255464...|       0.0|
|[25.0,9672.03,0.0...|    0|[4.43559397326021...|[0.98829070562264...|       0.0|
|[26.0,8939.61,0.0...|    0|[6.15008400873782...|[0.99787123874535...|       0.0|
|[29.0,9617.59,0.0...|    0|[4.23305900783083...|[0.98569952774234...|       0.0|
|[29.0,10203.18,1....|    0|[3.48723317919125...|[0.97032232352352...|       0.0|
|[29.0,12711.15,0....|    0|[5.02717091539803...|[0.99348538291962...|       0.0|
|[30.0,6744.87,0.0...|    0|[3.42850944426929...|[0.96858374288578...|       0.0|
|[30.0,8677.28,1.0...|    0|[3.85755892919764...|[0.97931731691272...|       0.0|
|[30.0,8874.83,0.0...|    0|[3.02997654308651...|[0.95391014167681...|       0.0|
|[30.0,10960.52,

In [33]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Churn')

In [34]:
Auc = churn_eval.evaluate(pred_and_lable.predictions)

In [35]:
Auc

0.7744186046511629

In [36]:
final_model = model.fit(final_data)

In [37]:
new_customers = spark.read.csv('/Users/eslinkiran/Apps/Pyspark-Tutorial/Logestic/new_customers.csv', inferSchema=True, header=True)

In [38]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [39]:
test_new_customer = assembler.transform(new_customers)

In [42]:
test_new_customer.show()

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+
|         Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|         Company|            features|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+
| Andrew Mccall|37.0|       9935.53|              1| 7.71|      8.0|2011-08-29 18:37:54|38612 Johnny Stra...|        King Ltd|[37.0,9935.53,1.0...|
|Michele Wright|23.0|       7526.94|              1| 9.28|     15.0|2013-07-22 18:19:54|21083 Nicole Junc...|   Cannon-Benson|[23.0,7526.94,1.0...|
|  Jeremy Chang|65.0|         100.0|              1|  1.0|     15.0|2006-12-11 07:48:13|085 Austin Views ...|Barron-Robertson|[65.0,100.0,1.0,1...|
|Megan Ferguson|32.0|        6487.5|              0|  9.4|     14.0|2016-10-28 05:32:13|922 Wright Branch...|   

In [44]:
final_result = final_model.transform(test_new_customer)

In [48]:
final_result.select('Company', 'prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

